In [1]:
from postgres import Postgres
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import numpy as np
import ast
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.cluster.hierarchy import dendrogram
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

pg = Postgres()

results = pg.query(
    """
 SELECT 
	cmetadata->'text' as text,
	embedding
    FROM langchain_pg_embedding 
	WHERE cmetadata @> '{"chunker": "sherpa"}' 
    AND LENGTH(cmetadata->>'text'::TEXT) > 40
	LIMIT 5000;
                   
	"""
)


df = pd.DataFrame(results, columns=["text", "embedding"])
df["embedding"] = df["embedding"].apply(ast.literal_eval)
print(type(df["embedding"][0]))
embeddings_array = np.array(df["embedding"].tolist())
print(embeddings_array.shape)

<class 'list'>
(2800, 1536)


,text,embedding
0,Introduction,"[-0.01311813099129231, -0.03373433119564931, 0..."
1,National and Local Planning Context,"[-0.0017892545784752606, -0.028909997633742576..."
2,The Parish of West Wittering,"[-0.003959975990752514, -0.014515406848524297,..."
3,Vision & Objectives,"[-0.007655357440304239, -0.02439560581313669, ..."
4,Policies & Proposals,"[0.01715402574925511, -0.03274377222368811, 0...."


In [6]:
tsne = TSNE(n_components=50, random_state=42)
embeddings_array = np.array(df["embedding"].tolist())
reduced_embeddings = tsne.fit_transform(embeddings_array)

# Standardize the data
scaler = StandardScaler()
reduced_embeddings = scaler.fit_transform(reduced_embeddings)

df["reduced_embedding"] = list(reduced_embeddings)


# Clustering
dbscan = DBSCAN(eps=eps_value, min_samples=min_samples_value)
clusters = dbscan.fit_predict(reduced_embeddings)
df["cluster"] = clusters

# Print the number of clusters found
num_unique_clusters = len(set(clusters))
print(f"Number of clusters found: {num_unique_clusters}")

df["tokens"] = df["text"]

# Compute word frequencies for each cluster
for cluster in range(1000):
    cluster_df = df[df["cluster"] == cluster]
    if not cluster_df.empty:
        cluster_tokens = df[df["cluster"] == cluster]["tokens"].explode()
        word_freq = Counter(cluster_tokens)
        most_common_words = word_freq.most_common(10)  # Top 10 words
        print(f"Cluster {cluster} Top Words:")
        for word, freq in most_common_words:
            print(f"{word}: {freq}")

        sample_docs = (
            df[df["cluster"] == cluster]["text"].sample(5, replace=True).tolist()
        )
        for doc in sample_docs:
            print(f"- {doc}")
        print("\n")

        print("\n")

ValueError: 'n_components' should be inferior to 4 for the barnes_hut algorithm as it relies on quad-tree or oct-tree.